In [14]:
import wandb
import pandas as pd
from PIL import Image, ImageOps
import json
import numpy as np


with wandb.init(project='diameterY', mode='online') as run:
    artifact = wandb.Artifact('rendered-fibers-mini-preview', type='table-dataset', description='rendered-fibers in table version')
    # run.use_artifact('rendered-fibers-table:latest')
    # artifact.download()
    df = pd.DataFrame(columns=['image', 'diameters', 'p1(s)', 'p2(s)', 'set'])
    for set_name, size in {'train':3,'val':3,'test':3}.items():
        for i in range(size):
            basename = f'../output/{set_name}{i:04d}'
            image = Image.open(f'{basename}.png')
            image = ImageOps.grayscale(image)
            image = np.array(image)
            # print(image.shape)
            seg = np.squeeze(np.load(f'{basename}_seg.npz')['y'])
            seg = np.max([np.zeros_like(seg, dtype=np.int32), seg.astype(np.int32)-99], axis=0).astype(int)
            with open(f'{basename}.json') as file:
                params = json.load(file)
            
            class_labels = {int(k)-99:f'f{int(k)-99:02d}' for k in params.keys() if k!='0'}
            class_labels.update({0:'bg'})
            wandb_im = wandb.Image(image, caption=basename, masks={
                'ground_truth':{
                    'mask_data': seg,
                    'class_labels': class_labels  
                }
            })
            vals = list(params.values())[1:]
            df = df.append({
                'image':wandb_im,
                'diameters': [v['d'] for v in vals],
                'p1(s)': [v['p1'] for v in vals],
                'p2(s)': [v['p2'] for v in vals],
                'set':set_name
            }, ignore_index=True)
    
    run.log({'preview_table':wandb.Table(dataframe=df)})
            




{1: 'f01', 2: 'f02', 3: 'f03', 4: 'f04', 5: 'f05', 6: 'f06', 7: 'f07', 8: 'f08', 9: 'f09', 10: 'f10', 11: 'f11', 12: 'f12', 13: 'f13', 14: 'f14', 15: 'f15', 16: 'f16', 17: 'f17', 18: 'f18', 19: 'f19', 20: 'f20', 21: 'f21', 22: 'f22', 23: 'f23', 24: 'f24', 25: 'f25', 26: 'f26', 27: 'f27', 28: 'f28', 29: 'f29', 30: 'f30', 31: 'f31', 32: 'f32', 33: 'f33', 34: 'f34', 35: 'f35', 36: 'f36', 37: 'f37', 38: 'f38', 39: 'f39', 40: 'f40', 41: 'f41', 42: 'f42', 43: 'f43', 44: 'f44', 45: 'f45', 46: 'f46', 47: 'f47', 48: 'f48', 49: 'f49', 50: 'f50', 0: 'bg'}
{1: 'f01', 2: 'f02', 3: 'f03', 4: 'f04', 5: 'f05', 6: 'f06', 7: 'f07', 8: 'f08', 9: 'f09', 10: 'f10', 11: 'f11', 12: 'f12', 13: 'f13', 14: 'f14', 15: 'f15', 16: 'f16', 17: 'f17', 18: 'f18', 19: 'f19', 20: 'f20', 21: 'f21', 22: 'f22', 23: 'f23', 24: 'f24', 25: 'f25', 26: 'f26', 27: 'f27', 28: 'f28', 29: 'f29', 30: 'f30', 31: 'f31', 32: 'f32', 33: 'f33', 34: 'f34', 35: 'f35', 36: 'f36', 37: 'f37', 38: 'f38', 39: 'f39', 40: 'f40', 41: 'f41', 42: 'f

/tmp/ipykernel_905982/2754839619.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/tmp/ipykernel_905982/2754839619.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/tmp/ipykernel_905982/2754839619.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/tmp/ipykernel_905982/2754839619.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/tmp/ipykernel_905982/2754839619.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/tmp/ipykernel_905982/2754839619.py:35: 

{1: 'f01', 2: 'f02', 3: 'f03', 4: 'f04', 5: 'f05', 6: 'f06', 7: 'f07', 8: 'f08', 9: 'f09', 10: 'f10', 11: 'f11', 12: 'f12', 13: 'f13', 14: 'f14', 15: 'f15', 16: 'f16', 17: 'f17', 18: 'f18', 19: 'f19', 20: 'f20', 21: 'f21', 22: 'f22', 23: 'f23', 24: 'f24', 25: 'f25', 26: 'f26', 27: 'f27', 28: 'f28', 29: 'f29', 30: 'f30', 31: 'f31', 32: 'f32', 33: 'f33', 0: 'bg'}
{1: 'f01', 2: 'f02', 3: 'f03', 4: 'f04', 5: 'f05', 6: 'f06', 7: 'f07', 8: 'f08', 9: 'f09', 10: 'f10', 11: 'f11', 12: 'f12', 13: 'f13', 14: 'f14', 15: 'f15', 16: 'f16', 17: 'f17', 18: 'f18', 0: 'bg'}
{1: 'f01', 2: 'f02', 3: 'f03', 4: 'f04', 5: 'f05', 6: 'f06', 7: 'f07', 8: 'f08', 9: 'f09', 10: 'f10', 11: 'f11', 12: 'f12', 13: 'f13', 0: 'bg'}
{1: 'f01', 2: 'f02', 3: 'f03', 4: 'f04', 5: 'f05', 6: 'f06', 7: 'f07', 8: 'f08', 9: 'f09', 10: 'f10', 11: 'f11', 12: 'f12', 13: 'f13', 14: 'f14', 15: 'f15', 16: 'f16', 17: 'f17', 18: 'f18', 19: 'f19', 20: 'f20', 21: 'f21', 22: 'f22', 23: 'f23', 24: 'f24', 25: 'f25', 26: 'f26', 0: 'bg'}


/tmp/ipykernel_905982/2754839619.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/tmp/ipykernel_905982/2754839619.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/tmp/ipykernel_905982/2754839619.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
